In [ ]:
import cv2
import argparse
import numpy as np
import os.path
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib import pyplot as plt
%matplotlib inline

from bokeh.plotting import figure #pip install bokeh
from bokeh.io import output_notebook, show, push_notebook


output_notebook()


def imshow(tit, image) :
    plt.title(tit)    
    if len(image.shape) == 3 :
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    else :
        plt.imshow(image, cmap="gray")
    plt.show()
    
    
def create_win(frames, scale=1.0) :    
    global myImage
    
    all = []
    for f in frames :
        if len(f.shape ) !=  3 : f = cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        all.append(f)
    frame = np.vstack(all)
    
    fr=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
    fr=cv2.flip(fr, -1) # because Bokeh flips vertically
    width=fr.shape[1]
    height=fr.shape[0]    

    p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=int(width*scale), height=int(height*scale))    
    myImage = p.image_rgba(image=[fr], x=0, y=0, dw=width, dh=height)
    show(p, notebook_handle=True)   
    
    
def update_win(frames) :
    
    all = []
    for f in frames :
        #print(len(f.shape))
        if len(f.shape ) !=  3 : f = cv2.cvtColor(f, cv2.COLOR_GRAY2BGR)
        all.append(f)
    frame = np.vstack(all)
    
    fr=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
    fr=cv2.flip(fr, 0)
    myImage.data_source.data['image']=[fr]
    push_notebook()
    # Initialize the parameters
confThreshold = 0.3  #Confidence threshold, 오브젝트일 확률
nmsThreshold = 0.4   #Non-maximum suppression threshold , 거리가 가까운애들은 하나의 그룹으로 보겟다, 주변보다 상대적으로 값이 높거나 낮거나 하는 애를 뽑겠다.

#YOLO v3는 영상을 무조건 416 사이즈로 변경
inpWidth = 416       #Width of network's input image
inpHeight = 416      #Height of network's input image

# Load names of classes
classesFile = "cfg/one.names" 
classes = None
with open(classesFile, 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')
print(classes)




# Get the names of the output layers
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames() #네트웍의 모든 이름을 가져오는 함수 ,총 갯수는 254개, 실제 레이어는 100 몇개,
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()] #실제 output 위치는 getUnconnectedOutLayers에서 1뺌

# Draw the predicted bounding box
#사각형으로 바운딩한 곳을 표시
def drawPred(frame, classId, conf, left, top, right, bottom): #클래스ID, 클래스에 대한 일치 확률,사각형 정보
    # Draw a bounding box.
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    # Get the label for the class name and its confidence
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

# Remove the bounding boxes with low confidence using non-maxima suppression
#
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only the
    # ones with high confidence scores. Assign the box's class label as the class with the highest score.
    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:] #확률 80개의 값을 가져옴
            classId = np.argmax(scores) #확률에서 높은거 가져옴
            confidence = scores[classId] #확률 값 가져옴
            if confidence > confThreshold: # confThreshold보다 더 높을 경우, 박스를 만듬
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
        
    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    #같은 포인트에서 다수의 바운딩 박스가 생성되어 있는걸 좀더 정확하게 판별해서 최소화 해줌
    
    for i in indices:
        i = i[0] #2차원 행렬이므로 
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(frame, classIds[i], confidences[i], left, top, left + width, top + height)
        
# Give the configuration and weight files for the model and load the network using them.
modelConfiguration = "cfg/one.cfg" #네트워크 구조체(CNN 필터 값들)가 포함되어있다, #컨피그 상에서 실제로 Yolo 관련 컨피그가 3개가 있다.
modelWeights = "cfg/one_class_v1_1800.weights"
        

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
        
# print("YOLO Video process")
# cap = cv2.VideoCapture("static/videos/swoon.mp4")
# frame_width = int(cap.get(3))
# frame_height = int(cap.get(4))
# print('width :%d, height : %d' % (frame_width, frame_height))
# fourcc = cv2.VideoWriter_fourcc(*'XVID') #운영체제별로 지원하는게 다름 확인필요
# out = cv2.VideoWriter('static/video_result/swoon.avi', fourcc, 29.0, (frame_width, frame_height)) 
# i = 0
# while(True):
#     hasFrame, frame = cap.read()    # Read 결과와 frame
#     if not hasFrame:
#         break
#     #print(hasFrame)
#     #print(frame)
#     print(i)
#     if(hasFrame) :
#         if i > 3300 and i < 7250:
#             print(i, ': caputer ..')
#             blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
#             net.setInput(blob)
#             outs = net.forward(getOutputsNames(net))
#             postprocess(frame, outs)
#             #cv2.imshow('frame_color', frame)    # 컬러 화면 출력
#             out.write(frame)
#         if i == 300:
#             break
#     i += 1

# cap.release()
# cv2.destroyAllWindows()

In [ ]:
import time
print("YOLO Video process")
cap = cv2.VideoCapture("static/videos/swoon.mp4")
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
print('width :%d, height : %d' % (frame_width, frame_height))
fourcc = cv2.VideoWriter_fourcc(*'XVID') #운영체제별로 지원하는게 다름 확인필요
out = cv2.VideoWriter('static/video_result/swoon.avi', fourcc, 29.0, (frame_width, frame_height)) 
i = 0
while(True):
    hasFrame, frame = cap.read()    # Read 결과와 frame
    if not hasFrame:
        break
    #print(hasFrame)
    #print(frame)
    print(i)
    if(hasFrame) :
        if i > 1 and i < 7250:
            print(i, ': caputer ..')
            start_time = time.time()
            blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
            net.setInput(blob)
            outs = net.forward(getOutputsNames(net))
            postprocess(frame, outs)
            #cv2.imshow('frame_color', frame)    # 컬러 화면 출력
            out.write(frame)
            end_time = time.time()
            print(f"inference_time: {round(end_time-start_time,2)}")
        if i == 300:
            break
    i += 1

cap.release()
cv2.destroyAllWindows()